In [53]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import random
random.seed(577)

import numpy as np
np.random.seed(577)
import pandas as pd

import torch
torch.set_default_tensor_type(torch.FloatTensor)
torch.use_deterministic_algorithms(True)
torch.manual_seed(577)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



import torch.optim as optim

from Module.gm_loss import GM_Loss
from Models.cnn import CNN

from torch.utils.data import DataLoader, random_split


import torch.nn as nn

from Module.util import compute_metrics

from torch.utils.data import DataLoader, TensorDataset

import warnings  
warnings.filterwarnings("ignore")


#plot
import matplotlib.pyplot as plt

In [54]:
data = pd.read_csv("Dataset/CreditCard/creditcard.csv")
X = data.iloc[:, data.columns != 'Class']
y = data.iloc[:, data.columns == 'Class']

In [55]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets (80% train, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Split the temporary set into validation and test sets (50% validation, 50% test of the 20% temp set)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Print the shapes of the splits to verify
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Training set: (227845, 30), (227845, 1)
Validation set: (28481, 30), (28481, 1)
Test set: (28481, 30), (28481, 1)


In [56]:
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [60]:
# Set parameters and initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = CNN(input_channels=1, input_size=input_size)
criterion = GM_Loss(with_logits=True)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 1000
batch_size = 32

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Training loop with batches
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        outputs = model(X_batch)
                print("hello")
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    epoch_loss /= len(train_loader)
    
    if (epoch+1) % 100 == 0:
        # Training metrics
        with torch.no_grad():
            train_outputs = model(X_train_tensor).view(-1)
            train_accuracy, train_f1, train_gm = compute_metrics(train_outputs, y_train_tensor)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor).view(-1)
            val_loss = criterion(val_outputs, y_val_tensor)
            val_accuracy, val_f1, val_gm = compute_metrics(val_outputs, y_val_tensor)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, '
              f'Train Accuracy: {train_accuracy:.4f}, Train F1 Score: {train_f1:.4f}, Train GM: {train_gm:.4f}, '
              f'Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}, '
              f'Validation F1 Score: {val_f1:.4f}, Validation GM: {val_gm:.4f}')
        
# Testing loop
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor).view(-1)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_accuracy, test_f1, test_gm = compute_metrics(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy:.4f}, '
          f'Test F1 Score: {test_f1:.4f}, Test GM: {test_gm:.4f}')

# Save the model (optional)
torch.save(model.state_dict(), 'fnn_model.pth')

hello


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1920 and 960x64)